AuxTel test LTS-337-030 (Auxiliary Telescope Dome Rotation Requirement)

In this notebook, we check the speed of the dome rotation. As part of the dome refurbishment effort, the minimum dome rotation speed shall be increased to a minimum of Aux_Tel_dome_rot_spd. The contractor shall supply the required hardware to support 4 drive motors. 

| Description | Value       | Unit          |   Name     |
| :---        |    :----:   |       :----:  |       ---: |
|The azimuth rotational speed for the Auxiliary Telescope Dome shall be a minimum of: | 4.0       | degrees/sec   |Aux_Tel_dome_rot_spd|

An angular rotation of three different lengths will be timed and results will get printed and stored in a text file (Attached AuxTel_LTS-337-030.txt) 

# Setting up notebook and ATCS system. 

In [1]:
import sys, time, os, asyncio
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [2]:
# Get EFD client
client = EfdClient('summit_efd')

In [3]:
domain = salobj.Domain()

In [4]:
atcs = ATCS(domain)
await asyncio.gather(atcs.start_task)

[[None, None, None, None, None, None, None]]

In [5]:
# Enable components ATCS
await atcs.enable()

In [8]:
# Create the EFD logger object
index=os.getuid()
c = salobj.Controller(name="Test", index=index)

In [9]:
await c.start_task

# Test Long Rotation

In [10]:
# Disable Dome following 
await atcs.disable_dome_following()

In [12]:
# Enter below the number of test iterations 
# and initial and final dome azimuth positions.
number_of_tests = 1
filename = 'AuxTel_LTS-337-030_20220304.txt'

# Long Rotation 180 degrees
test_type = 'Long Rotation - 180 deg'
initial_position = 0
final_position = 180

# # Medium rotation - 50 degrees
# test_type = 'Medium Rotation - 50 deg'
# initial_position = 0
# final_position = 50

# # Short rotation - 10 degrees
# test_type = 'Short Rotation - 25 deg'
# initial_position = 0
# final_position = 20

print(f'The initial position is {initial_position} degrees \n'
      f'The final position is {final_position} degrees \n'
      f'The number of test iterations is {number_of_tests}\n'
      f'Results will be saved in {filename}')

The initial position is 0 degrees 
The final position is 180 degrees 
The number of test iterations is 1
Results will be saved in AuxTel_LTS-337-030_20220304.txt


In [34]:
start_end_values = []

n = 1
while n < number_of_tests + 1: 
    
    c.log.info(f'Starting Dome Rotation Requirement test iteration {n} '
               f'on {time.asctime()}')

    # Move to initial position 
    print(f'Moving to initial azimuth position = '
          f'{initial_position} degrees on {time.asctime()}')

    await atcs.slew_dome_to(az=initial_position)

    # Get dome azimuth initial position and move start time 
    c.log.info(f'Move Start Time at {time.asctime()}')
    start_time = Time(Time.now(), format='fits', 
                             scale='tai')
    
    dome_azimuth = await client.select_top_n(
        "lsst.sal.ATDome.position", 'azimuthPosition', 1)
    az_origin = dome_azimuth.iat[0,0]
    print(f'Initial azimuth position is = {az_origin} degrees '
          f'on {start_time}')

    # Move to final position
    print(f'Moving to final azimuth position = '
          f'{final_position} degrees on {time.asctime()}')
    await atcs.slew_dome_to(az=final_position)
 
    
    # Get dome azimuth final position and move end time
    c.log.info(f'Move End Time at {time.asctime()}')
    end_time = Time(Time.now(), format='fits', scale='tai')
    
    dome_azimuth = await client.select_top_n(
        "lsst.sal.ATDome.position", 'azimuthPosition', 1)
    az_final = dome_azimuth.iat[0,0]
    print(f'Final azimuth position is = {az_final} degrees ' 
          f'on {end_time}')


    # Some quick calculations and print results. 
    duration_time= TimeDelta(end_time - start_time, 
                             format='sec')
    rotation_angle = abs(az_origin - az_final)

    speed = rotation_angle/duration_time

    print(f'#########################################################\n'
          f'The move duration was {duration_time.value:.2f} seconds \n'
          f'for an angular rotation of {rotation_angle:.2f} degrees \n'
          f'with a rotational speed of {speed.value:.2f} degrees/sec\n'
          f'#########################################################')
    
    # Store the test values in a list. 
    pair_times = [start_time.value, end_time.value, 
                  round(rotation_angle,2),round(speed.value,2)]
    start_end_values.append(pair_times)
    
    n=n+1

# Save test results in a file (See file attached)
with open(filename, 'a') as f:
    f.write('{} test with {} iteration on {} \n'.format(test_type, n-1, time.asctime()))
    f.write('Start Time, End Time, Rotation angle (degrees), ' 
            'Rotational speed (degrees/sec) \n'.format(n, time.asctime()))
    for item in start_end_values:
        f.write("{}\n".format(item))
f.close()
print(f'Start and end move times, rotation angle (degrees) and rotational speed' 
      f'(degrees/sec) are recorded in {filename}')

Moving to initial azimuth position = 0 degrees on Mon Jan 31 19:32:06 2022


Initial azimuth position is = 359.68 degrees on 2022-01-31T19:33:06.274
Moving to final azimuth position = 180 degrees on Mon Jan 31 19:32:29 2022


Final azimuth position is = 184.49 degrees on 2022-01-31T19:34:01.393
#########################################################
The move duration was 55.12 seconds 
for an angular rotation of 175.19 degrees 
with a rotational speed of 0.31 degrees/sec
#########################################################


Moving to initial azimuth position = 0 degrees on Mon Jan 31 19:33:24 2022


Initial azimuth position is = 359.26 degrees on 2022-01-31T19:35:07.115
Moving to final azimuth position = 180 degrees on Mon Jan 31 19:34:30 2022


Final azimuth position is = 180.05 degrees on 2022-01-31T19:36:12.835
#########################################################
The move duration was 65.72 seconds 
for an angular rotation of 179.21 degrees 
with a rotational speed of 0.37 degrees/sec
#########################################################
Start and end move times, rotation angle (degrees) and rotational speed(degrees/sec) are recorded in AuxTel_LTS-337-030.txt


# Results

In [35]:
print(f'######################################################### \n'
          f'The move duration was {duration_time.value:.2f} seconds \n'
          f'for an angular rotation of {rotation_angle:.2f} degrees \n'
          f'with a rotational speed of {speed.value:.2f} degrees/sec\n'
          f'#########################################################')
    

######################################################### 
The move duration was 65.72 seconds 
for an angular rotation of 179.21 degrees 
with a rotational speed of 0.37 degrees/sec
#########################################################


# Shutdown

In [37]:
# Back in standby.
await atcs.shutdown()